In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
import numpy as np


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Quadro M6000 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN not available)


In [2]:
filename = "allLyrics.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
chars = set(raw_text)

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


maxlen = 100
step = 1
sentences = []
next_chars = []
for i in range(0, len(raw_text) - maxlen, step):
    sentences.append(raw_text[i: i + maxlen])
    next_chars.append(raw_text[i + maxlen])
print('nb sequences:', len(sentences))



X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

('nb sequences:', 283856)


In [3]:
model = Sequential()
model.add(LSTM(512,  input_shape=(100,len(chars)), return_sequences=True))
model.add(Dropout(0.20))

# model.add(LSTM(512, return_sequences=True))
# model.add(Dropout(0.20))

# model.add(LSTM(512, return_sequences=True))
# model.add(Dropout(0.20))

model.add(LSTM(256, return_sequences=False))
model.add(Dropout(0.20))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

# compile or load weights then compile depending


In [4]:
# "This above all: to thine own self be true". - Hamlet (Act I, Scene III).
def generatetext(i):
    seed_text = "I'll be the fire in your night. then it's love you take. I will defend, I will fight. I'll be there when you need me. When honor's at stake,".lower()
    generated = '' + seed_text[-100:]
    print(i,"Seed",seed_text)
    for iteration in range(300):
            # create x vector from seed to predict off of
            x = np.zeros((1, 100, len(chars)))
            for t, char in enumerate(seed_text[-100:]):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = np.argmax(preds)
            next_char = indices_char[next_index]

            generated += next_char
            seed_text = seed_text[1:] + next_char
#             print seed_text,next_char
    print('follow up with: ' + generated)

In [5]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

In [15]:
# model.load_weights('One_go_larger_network_0.h5')

In [8]:
epochs = 30
for i in range(10,epochs):
    model.fit(X,y, batch_size=1000,nb_epoch=1)
    model.save_weights('lyrics_layers'+str(i)+'.h5')
    generatetext(i)

Epoch 1/1
283856/283856 [==============================] - 353s - loss: 1.0049 - acc: 0.6930   
(10, 'Seed', "i'll be the fire in your night. then it's love you take. i will defend, i will fight. i'll be there when you need me. when honor's at stake,")
follow up with: s love you take. i will defend, i will fight. i'll be there when you need me. when honor's at stake, i'll be there
i will be there i will be there

i will be there i will be there

i wanna hold you back to me

i wanna hold you bore
this words i can say
i love you so much to see you are all i have to say you love me to be when i couldn't see
you say it best when i thought i can say
i lov
Epoch 1/1
283856/283856 [==============================] - 353s - loss: 0.9494 - acc: 0.7107   
(11, 'Seed', "i'll be the fire in your night. then it's love you take. i will defend, i will fight. i'll be there when you need me. when honor's at stake,")
follow up with: s love you take. i will defend, i will fight. i'll be there when you nee

283856/283856 [==============================] - 353s - loss: 0.6562 - acc: 0.7993   
(23, 'Seed', "i'll be the fire in your night. then it's love you take. i will defend, i will fight. i'll be there when you need me. when honor's at stake,")
follow up with: s love you take. i will defend, i will fight. i'll be there when you need me. when honor's at stake, they'll gave it all for one and all for love.

[boy:]
i can do that
i can do that

[girl:]
i can say it our love will lead me to be
the way that i want to be

those you won't be there
i'll be the walking for love
i'll be the heaven love was more than walking,
and i know that i'm here t
Epoch 1/1
283856/283856 [==============================] - 353s - loss: 0.5379 - acc: 0.8316   
(24, 'Seed', "i'll be the fire in your night. then it's love you take. i will defend, i will fight. i'll be there when you need me. when honor's at stake,")
follow up with: s love you take. i will defend, i will fight. i'll be there when you need me. when h

In [ ]:
generatetext(i)
